# Initialize

In [1]:
import pynq
from pynq.overlays.base import BaseOverlay
from ctypes import *
import time
import numpy as np

ol = BaseOverlay("base.bit")

# PID

In [6]:
%%microblaze ol.PMODA

#define MOTOR_COUNT 6
#define RC_CHANNELS 6
#define clip(in,low,high) ((in)<(low)?(low):((in)>(high)?(high):(in)))
float MIX_C[6][3] = {{.5,-0.57735026919,1},{1,0,-1},{.5,.57735026919,1},{-.5,-.57735026919,-1},{-1,0,1},{-.5,.57735026919,-1}};

void pid(float rcCmdIn[5],float measured[4],float kp[3],float kd[2], float ki[2],float commandOut[4096]) {

static float last_error[2]={0,0};
static float integral[2]={0,0};
float pid_o[3];
float curr_error[2];
float deriv[2];
float correction[2];
#****************************************
#ROLL PID CONTROLLER
#*****************************************/

curr_error[0]= rcCmdIn[0] - measured[0]*5;
integral[0] =  clip(float(integral[0] + curr_error[0]),float(-100),float(100));
deriv[0] = (curr_error[0]-last_error[0]);
correction[0] = (kp[0] * curr_error[0]) + (ki[0] * integral[0]) + (kd[0] * deriv[0]);
pid_o[0] = clip(correction[0],float(-1),float(.999));
last_error[0] = curr_error[0];

#****************************************
#pitch PID CONTROLLER
#*****************************************/

curr_error[1] = rcCmdIn[1] - measured[1]*5;
integral[1] =  clip(float(integral[1] + curr_error[1]),float(-100),float(100));
deriv[1] = (curr_error[1]-last_error[1]);
correction[1] = (kp[1] * curr_error[1]) + (ki[1] * integral[1]) + (kd[1] * deriv[1]);
pid_o[1] = clip(correction[1],float(-1),float(.999));
last_error[1] = curr_error[1];

#****************************************
#yaw P CONTROLLER
#****************************************

pid_o[2] = kp[2]*(rcCmdIn[3] - measured[3]);


# mixed _in contains noramlized values for each channel
# lets convert those to what we want to use
# change all to float and make sure thrust is scaled to [0,1)
float r_c = pid_o[0];
float p_c = pid_o[1];
float t_c = rcCmdIn[2]*float(.5)+float(.5);//move scale to [-.5,.5) then [0,1)
float y_c = pid_o[2];

for(char i=0; i < 6; i++){
float scaled_power = t_c+(r_c*MIX_C[i][0]+p_c*MIX_C[i][1]+ y_c*MIX_C[i][2])*float(.33);
commandOut[i]=float(clip(scaled_power,float(0),float(.999)));
}
commandOut[6]=rcCmdIn[4];
}